In [ ]:
with open('final_sen_v2.txt', 'r') as f:
  sen_v2 = f.read()

sen_v2 = sen_v2.split('\n')

In [ ]:
class TFIDF:
  def __init__(self, a1):
    self.docs = a1
    self.vectorizer = TfidfVectorizer(use_idf = True, norm ='l2', ngram_range=(1,1), analyzer='word')
    self.vectorizer.fit_transform(self.docs)
    self.stopwords = [x.strip() for x in codecs.open('stopwords.txt','r','utf-8').readlines()]
  
  def forward(self, querry, k=10, is_vector=False, words=[]):
    ten_tokens = [0] * k
    ten_scores = [-100] * k

    if not is_vector:
      vector = self.vectorizer.transform([querry])
      b = vector.toarray()
      q1 = querry.split()
    else:
      vector = querry
      b = [vector.tolist()]
      q1 = words
      # print(words)
      


    NR = []

    # print(len(b[0]))
    
    for item in tqdm(self.docs):
      is_common = False
      items = item.split()
      for itemq in q1:
        if itemq in items:
          is_common = True
      if not is_common:
        NR.append(item)
        continue
      vector = self.vectorizer.transform([item])
      c = vector.toarray()
      if (norm(c[0])*norm(b[0])) != 0:
        cos_sim = dot(c[0], b[0])/(norm(c[0])*norm(b[0]))
        if cos_sim != 0:
          # print(cos_sim)
          # print(item)
          for i in range(k):
            if ten_scores[i] < cos_sim:
              ten_scores[i] = cos_sim
              ten_tokens[i] = item
              break
    # return ten_scores, ten_tokens
    return ten_tokens, NR

  def query_expansion(self, R, NR, query):
    NR = random.sample(NR, 10)
    R_vecs = []
    NR_vecs = []
    words = []
    for item in R:
      items = item.split()
      for itemq in items:
        if itemq not in self.stopwords:
          words.append(itemq)
      vector = self.vectorizer.transform([item])
      c = np.array(vector.toarray()[0])
      R_vecs.append(c)
    words = random.sample(words, 6)
    for oo in query.split():
      words.append(oo)
    R_vecs = np.array(R_vecs)
    R_mean = np.mean(R_vecs, axis=0)
    for item in NR:
      vector = self.vectorizer.transform([item])
      c = np.array(vector.toarray()[0])
      NR_vecs.append(c)
    NR_vecs = np.array(NR_vecs)
    NR_mean = np.mean(NR_vecs, axis=0)
    # out1, _ = self.forward(query, 30)
    vector = R_vecs[-1]
    # vector = self.vectorizer.transform(out1)
    R_expanded = vector + 1.5 * R_mean - 0.1 * NR_mean
    
    en_tokens = 0
    en_scores = -100

    b = R_expanded
    for i in range(len(R_vecs)):
        c = R_vecs[i]
        if (norm(c)*norm(b)) != 0:
          cos_sim = dot(c, b)/(norm(c)*norm(b))
          if cos_sim != 0:
            if en_scores < cos_sim:
              # print(1111111, en_scores, cos_sim)
              en_scores = cos_sim
              en_tokens = R
              
    return en_tokens

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from numpy import dot
from numpy.linalg import norm

from nltk import FreqDist
import itertools
import pandas as pd
import codecs
# from hazm import *
import torch

from tqdm import tqdm

import random

import numpy as np
import codecs

In [ ]:
b1 = TFIDF(sen_v2)
q = 'کزین برتر'
R, NR = b1.forward(q)
expanded_query = b1.query_expansion(R, NR, q)[0]
